# Generate the Figures in the Paper

In [ ]:
import numpy as np

import os
import sklearn
from interpret.glassbox import ExplainableBoostingClassifier

from itertools import product

import matplotlib.pyplot as plt
import seaborn as sns

import datasets
import paperutil
import nshap

%load_ext autoreload
%autoreload 2

### Load the computed n-Shapley Values

In [ ]:
data_sets = ['folk_income', 'folk_travel', 'housing', 'credit', 'iris']
classifiers = ['gam', 'rf', 'gbtree', 'knn']
methods = ['predict', 'proba', 'decision']

In [ ]:
shapley_values = {}
for dataset in data_sets:
    X_train, X_test, _, _, _ = datasets.load_dataset(dataset)
    shapley_values[dataset] = {}
    num_datapoints = min(5000, X_train.shape[0]) 
    for classifier in classifiers:
        shapley_values[dataset][classifier] = {}
        for method in methods:
            if os.path.exists(f'../../results/n_shapley_values/{dataset}/{classifier}/observation_0_{method}_{num_datapoints}.JSON'):
                shapley_values[dataset][classifier][method] = []
                for i_datapoint in range(min(X_test.shape[0], 100)):
                    fname = f'../../results/n_shapley_values/{dataset}/{classifier}/observation_{i_datapoint}_{method}_{num_datapoints}.JSON'
                    if os.path.exists(fname):
                        n_shapley_values = nshap.load(fname)
                        shapley_values[dataset][classifier][method].append(n_shapley_values)
                    else:
                        print(f'File {fname} not found.')

### Create directory structure

In [ ]:
paths = ['../../figures/', '../../figures/partial_dependence/', '../../figures/shapley_gam/', '../../figures/n_shapley_values/']
for p in paths:
    if not os.path.exists( p ):
        os.mkdir( p )

### Plot Settings

In [ ]:
# avoid type-3 fonts
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
rotation = {'folk_income': 60, 'folk_travel': 60, 'housing': 60, 'credit': 60, 'iris': 0}

In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook", rc={'axes.linewidth': 2, 'grid.linewidth': 1},  font_scale=2)

In [ ]:
def asdjust_ylim(axlist):
    ymin = min([x.get_ylim()[0] for x in axlist])
    ymax = max([x.get_ylim()[1] for x in axlist])
    for ax in axlist:
        ax.set_ylim((ymin, ymax))

### Plots of n-Shapley Values

In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook", rc={'axes.linewidth': 2, 'grid.linewidth': 1},  font_scale=2.1)

In [ ]:
for dataset in data_sets:
    feature_names = datasets.get_feature_names(dataset)
    for classifier in classifiers:
        for method in methods:
            # different methods for different classifiers
            if not method in shapley_values[dataset][classifier]: 
                continue
            print(dataset, classifier, method)
            for i_datapoint in range(5):
                n_shapley_values = shapley_values[dataset][classifier][method][i_datapoint]
                fig, ax = plt.subplots(1, 4, figsize=(22.5, 6.75))
                ax0 = nshap.plot_n_shapley(n_shapley_values.k_shapley_values(1), axis=ax[0], legend=False, feature_names=feature_names, rotation=rotation[dataset])
                ax0.set_ylabel('Feature Attribution')
                ax0.set_title('Shapley Values')
                ax1 = nshap.plot_n_shapley(n_shapley_values.k_shapley_values(2), axis=ax[1], legend=False, feature_names=feature_names, rotation=rotation[dataset])
                ax1.set(yticklabels= []) 
                ax1.set_title('Shapley Interaction Values')
                ax2 = nshap.plot_n_shapley(n_shapley_values.k_shapley_values(4), axis=ax[2], legend=False, feature_names=feature_names, rotation=rotation[dataset])
                ax2.set(yticklabels= [])
                ax2.set_title('4-Shapley Values')
                ax3 = nshap.plot_n_shapley(n_shapley_values, axis=ax[3], legend=False, feature_names=feature_names, rotation=rotation[dataset])
                ax3.set(yticklabels= []) 
                ax3.set_title('Shapley-GAM')
                axes = [ax0, ax1, ax2, ax3]
                ymin = min([x.get_ylim()[0] for x in axes])
                ymax = max([x.get_ylim()[1] for x in axes])
                for x in axes:
                    x.set_ylim((ymin, ymax))
                plt.tight_layout()
                plt.savefig(f'../../figures/n_shapley_values/{dataset}_{classifier}_{method}_{i_datapoint}.pdf')
                if i_datapoint == 0:
                    plt.show()
                plt.close(fig)

### Plots of n-Shapley Values in the Appendix

In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook", rc={'axes.linewidth': 2, 'grid.linewidth': 1},  font_scale=2.25)

from itertools import product

for dataset in data_sets:
    feature_names = datasets.get_feature_names(dataset)
    if dataset == 'iris':
        continue
    for classifier in classifiers:
        method = 'proba'
        if dataset == 'housing':
            method = 'predict'
        if classifier == 'gam':
            method = 'decision'
        print(dataset, classifier, method)
        for i_datapoint in range(1):
            n_shapley_values = shapley_values[dataset][classifier][method][i_datapoint]
            if dataset == 'housing': # 8 features
                ncols = 4
                fig, ax = plt.subplots(2, ncols, figsize=(26, 14.75))
            else: # 10 features
                ncols = 5
                fig, ax = plt.subplots(2, ncols, figsize=(32, 14.75))
            for i in range(2):
                for j in range(ncols):
                    k = 1 + ncols*i + j
                    nshap.plot_n_shapley(n_shapley_values.k_shapley_values(k), axis=ax[i, j], legend=False, feature_names=feature_names, rotation=rotation[dataset])
                    ax[i, j].set_title(f'{k}-Shapley Values')
            ax[0, 0].set_ylabel('Feature Attribution')
            ax[1, 0].set_ylabel('Feature Attribution')
            ax[0, 0].set_title('Shapley Values')
            ax[0, 1].set_title('Shapley Interaction Values')
            ax[1, ncols-1].set_title('Shapley-GAM')
            axes = [ax[i,j] for (i,j) in product(range(2), range(ncols))]
            asdjust_ylim(axes)
            for j in range(ncols):
                ax[0, j].set(xticklabels= [])
            for i in range(2):
                for j in range(1,ncols):
                    ax[i, j].set(yticklabels= [])
            plt.tight_layout()
            plt.savefig(f'../../figures/n_shapley_values/apx_{dataset}_{classifier}_{method}_{i_datapoint}_full.pdf')
            if i_datapoint == 0:
                plt.show()
            plt.close(fig)

### Example Visualizations

In [ ]:
values = {(i,):0 for i in range(4)}
values[(2,)] = 0.2
values[(3,)] = -0.1
n_shapley_values = nshap.nShapleyValues(values)
        
fig, ax = plt.subplots(1, 1, figsize=(5.5, 6))
nshap.plot_n_shapley(n_shapley_values.k_shapley_values(1), legend=False, axis=ax)
plt.tight_layout()
plt.savefig('../../figures/example1.pdf')
print(values)
plt.show()

values[(1,2)] = 0.1    
fig, ax = plt.subplots(1, 1, figsize=(5.5, 6))
nshap.plot_n_shapley(nshap.nShapleyValues(values), legend=False, axis=ax)
plt.tight_layout()
plt.savefig('../../figures/example2.pdf')
print(values)
plt.show()

values[(2,3)] = -0.1 
fig, ax = plt.subplots(1, 1, figsize=(5.5, 6))
nshap.plot_n_shapley(nshap.nShapleyValues(values), legend=False, axis=ax)
plt.tight_layout()
plt.savefig('../../figures/example3.pdf')
print(values)
plt.show()

values[(1,2,3)] = 0.1
fig, ax = plt.subplots(1, 1, figsize=(5.5, 6))
nshap.plot_n_shapley(nshap.nShapleyValues(values), legend=False, axis=ax)
plt.tight_layout()
plt.savefig('../../figures/example4.pdf')
print(values)
plt.show()

values[(0,1,2,3)] = -0.1    
fig, ax = plt.subplots(1, 1, figsize=(5.5, 6))
nshap.plot_n_shapley(nshap.nShapleyValues(values), legend=False, axis=ax)
plt.tight_layout()
plt.savefig('../../figures/example5.pdf')
print(values)
plt.show()

### The legend

In [ ]:
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
from matplotlib.transforms import Bbox

fig, ax = plt.subplots(1, 1, figsize=(12, 1))

# legend
plot_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', 
               '#17becf', 'black', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

color_patches = [mpatches.Patch(color=color) for color in plot_colors]
lables = ['Main']
lables.append('2nd order')
lables.append('3rd order')
for i in range(4, 10):
    lables.append(f'{i}th')
lables.append('10th order')
ax.legend(color_patches, lables, ncol=11, fontsize=30, handletextpad=0.5, handlelength=1, handleheight=1)
plt.axis('off')
plt.savefig(f'../../figures/legend.pdf', bbox_inches=Bbox([[-13.5, -0.2], [11, 0.9]]))
plt.show()

In [ ]:
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
from matplotlib.transforms import Bbox

fig, ax = plt.subplots(1, 1, figsize=(12, 1))

# legend
plot_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', 
               '#17becf', 'black', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

color_patches = [mpatches.Patch(color=color) for color in plot_colors]
lables = ['Main']
lables.append('2nd order')
lables.append('3rd order')
for i in range(4, 8):
    lables.append(f'{i}th')
ax.legend(color_patches, lables, ncol=11, fontsize=30, handletextpad=0.5, handlelength=1, handleheight=1)
plt.axis('off')
plt.savefig(f'../../figures/legend7.svg', bbox_inches=Bbox([[-13.5, -0.2], [11, 0.9]]))
plt.show()

### Shapley-GAM Figure in the paper

In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook", rc={'axes.linewidth': 2, 'grid.linewidth': 1},  font_scale=1.9)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5.5, 6))
ax = nshap.plot_n_shapley(shapley_values['credit']['gam']['decision'][0], axis=ax, legend=False, feature_names=datasets.get_feature_names('credit'), rotation=60)
ax.set_ylabel('Feature Attribution')
ax.set_title('Glassbox-GAM')
plt.tight_layout()
plt.savefig(f'../../figures/A.svg')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4.7, 6))
ax = nshap.plot_n_shapley(shapley_values['housing']['gbtree']['predict'][0], axis=ax, legend=False, feature_names=datasets.get_feature_names('housing'), rotation=60)
ax.set_title('Gradient Boosted Tree')
print(ax.get_ylim())
plt.tight_layout()
plt.savefig(f'../../figures/B.svg')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5.5, 6))
ax = nshap.plot_n_shapley(shapley_values['folk_travel']['knn']['proba'][1], axis=ax, legend=False, feature_names=datasets.get_feature_names('folk_travel'), rotation=60)
ax.set_title('k-Nearest Neighbor')
print(ax.get_ylim())
ax.set_ylim((-0.32, 0.34))
plt.tight_layout()
plt.savefig(f'../../figures/C.svg')
plt.show()

In [ ]:
n_shapley_values = {}
for S in nshap.powerset(range(8)):
    if len(S) == 0:
        continue
    elif len(S) == 8:
        n_shapley_values[S] = 1
    else:
        n_shapley_values[S] = 0 
n_shapley_values = nshap.nShapleyValues(n_shapley_values)

fig, ax = plt.subplots(1, 1, figsize=(4.7, 6))
ax = nshap.plot_n_shapley(n_shapley_values, axis=ax, legend=False, rotation=60)
ax.set_title('8d Checkerboard Function')
print(ax.get_ylim())
ax.set_ylim((-0.32, 0.34))
plt.yticks([]) 
plt.tight_layout()
plt.savefig(f'../../figures/D.svg')
plt.show()

### Partial dependence plots

In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook", rc={'axes.linewidth': 2, 'grid.linewidth': 1},  font_scale=2.6)

In [ ]:
for dataset in data_sets:
    feature_names = datasets.get_feature_names(dataset)
    X_train, X_test, _, _, _ = datasets.load_dataset(dataset)
    num_datapoints = min(5000, X_train.shape[0]) 
    for classifier in classifiers:
        method = 'proba'
        if dataset == 'housing':
            method = 'predict'
        if classifier == 'gam':
            method = 'decision'
        print(dataset, classifier, method)
        clf = paperutil.train_classifier(dataset, classifier)
        for i_feature in range(len(feature_names)):
            # collect data
            x = []
            nsv_list = []
            for i_datapoint in range(100):
                fname = f'../../results/n_shapley_values/{dataset}/{classifier}/observation_{i_datapoint}_{method}_{num_datapoints}.JSON'
                if os.path.exists(fname):
                    n_shapley_values = nshap.load(fname)
                    if method == 'proba': 
                        # we computed the shapley values for the probablity of the predicted class
                        # but here we want to explain the probability of class 1, for all data points
                        prediction = int( clf.predict( X_test[i_datapoint, :].reshape((1,-1)) ) )
                        x.append(X_test[i_datapoint, i_feature])
                        if prediction == 0: 
                            n_shapley_values = nshap.nShapleyValues({k:-v for k,v in n_shapley_values.items()})
                        nsv_list.append(n_shapley_values) 
                    else:
                        x.append(X_test[i_datapoint, i_feature])
                        nsv_list.append(n_shapley_values)
                else:
                    print(f'File {fname} not found')
            # plot
            fig, ax = plt.subplots(1, 4, figsize=(30, 5.5)) # appendix
            #fig, ax = plt.subplots(1, 4, figsize=(30, 6)) # paper
            y = [n_shapley_values.k_shapley_values(1)[(i_feature,)] for n_shapley_values in nsv_list]
            ax0 = sns.scatterplot(x=x, y=y, ax=ax[0], s=150)
            ax0.set_ylabel('Feature Attribution')
            ax0.set_title('Shapley Values')
            y = [n_shapley_values.k_shapley_values(2)[(i_feature,)] for n_shapley_values in nsv_list]
            ax1 = sns.scatterplot(x=x, y=y, ax=ax[1], s=150)
            ax1.set(yticklabels= []) 
            ax1.set_title('Shapley Interaction Values')
            y = [n_shapley_values.k_shapley_values(4)[(i_feature,)] for n_shapley_values in nsv_list]
            ax2 = sns.scatterplot(x=x, y=y, ax=ax[2], s=150)
            ax2.set(yticklabels= [])
            ax2.set_title('4-Shapley Values')
            y = [n_shapley_values[(i_feature,)] for n_shapley_values in nsv_list]
            ax3 = sns.scatterplot(x=x, y=y, ax=ax[3], s=150)
            ax3.set(yticklabels= []) 
            ax3.set_title('Shapley-GAM')
            axes = [ax0, ax1, ax2, ax3]
            asdjust_ylim(axes)
            for ax in axes:
                ax.set_xlabel(f'Value of Feature {feature_names[i_feature]}')
            plt.tight_layout()
            plt.savefig(f'../../figures/partial_dependence/{dataset}_{classifier}_{i_feature}_{method}.pdf')
            plt.show()
            plt.close(fig)

### Recovery of GAM without interaction terms

In [ ]:
from interpret.glassbox import ExplainableBoostingClassifier

In [ ]:
X_train, X_test, Y_train, Y_test, feature_names = datasets.load_dataset('folk_travel')

In [ ]:
ebm = ExplainableBoostingClassifier(feature_names=feature_names, interactions=0, random_state=0)
ebm.fit(X_train[:50000], Y_train[:50000])
(ebm.predict(X_test) == Y_test).mean()

In [ ]:
from interpret.provider import InlineProvider
from interpret import set_visualize_provider

set_visualize_provider(InlineProvider())

from interpret import show

ebm_global = ebm.explain_global()
show(ebm_global)

#### Compute KernelShap explanations

In [ ]:
import shap 

X_train_summary = shap.kmeans(X_train, 25)
kernel_explainer = shap.KernelExplainer(ebm.decision_function, X_train_summary)

In [ ]:
kernel_shap_values = []
for i in range(100):
    kernel_shap_values.append( kernel_explainer.shap_values(X_test[i, :]) )

In [ ]:
for ifeature in [1]:
    print(f'---------------------------  {ifeature} ---------------------')
    # partial influence of ifeature in the gam
    x_ifeature = []
    gam_v = []
    for i in range(100):
        x_hat = np.zeros((1,10))
        x_hat[0, ifeature] = X_test[i, ifeature]
        x_ifeature.append(X_test[i, ifeature])
        gam_v.append(ebm.decision_function(x_hat)[0])
    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    sns.scatterplot(x_ifeature, gam_v-np.mean(gam_v))
    plt.title('Explainable Boosting')
    plt.xlabel(f'{feature_names[ifeature]}')
    plt.ylabel(f'Score')
    plt.tight_layout()
    plt.savefig(f'../../figures/recovery_ebm.pdf')
    plt.show()

    # shapley value of feature i
    shapley_v = []
    for i in range(100):
        shapley_v.append(kernel_shap_values[i][ifeature])
    fig, ax = plt.subplots(1, 1, figsize=(6, 5))
    sns.scatterplot(x_ifeature, shapley_v-np.mean(shapley_v))
    plt.title('kernel SHAP')
    plt.xlabel(f'{feature_names[ifeature]}')
    plt.ylabel(f'Score')
    plt.tight_layout()
    plt.savefig(f'../../figures/recovery_kernel_shap.pdf')
    plt.show()

In [ ]:
sns.set_style("whitegrid")
sns.set_context("notebook", rc={'axes.linewidth': 2, 'grid.linewidth': 1},  font_scale=1.4)

img = plt.imread("../../figures/gam_curve.png")
fig, ax = plt.subplots(figsize=(7,4))
ax.imshow(img, extent=[-2.1, 2.15, -0.99, 0.83], aspect='auto')
sns.scatterplot(x_ifeature, shapley_v-np.mean(shapley_v), color='r', s=50)
plt.xlabel(f'Value of Feature POWPUMA')
plt.ylabel(f'Kernel SHAP Attribution')
plt.tight_layout()
plt.savefig(f'../../figures/recovery.svg')
plt.show()

### Accuracy vs Average Degree of Variable Interaction in the Shapley-GAM

In [ ]:
accuracies = {}
for dataset in data_sets:
    X_train, X_test, Y_train, Y_test, feature_names = datasets.load_dataset(dataset)
    is_classification = datasets.is_classification(dataset)
    accuracies[dataset] = {}
    for classifier in classifiers:
        clf = paperutil.train_classifier(dataset, classifier)
        # accuracy / mse
        if is_classification:
            accuracies[dataset][classifier] = sklearn.metrics.accuracy_score(Y_test, clf.predict(X_test))
        else:
            accuracies[dataset][classifier] = sklearn.metrics.mean_squared_error(Y_test, clf.predict(X_test))
        print(dataset, classifier, accuracies[dataset][classifier])

In [ ]:
complexities = {}
for dataset in data_sets:
    complexities[dataset] = {}
    for classifier in classifiers:
        method = 'proba'
        if dataset == 'housing':
            method = 'predict'
        if method == 'proba' and classifier == 'svm':
            continue
        if classifier == 'gam':
            method = 'decision'
        v = []
        for n_shapley_values in shapley_values[dataset][classifier][method]:  
            degree_contributions = n_shapley_values.get_degree_contributions()
            integral = np.sum(degree_contributions*list(range(1, len(degree_contributions)+1))) / np.sum(degree_contributions)
            v.append(integral)
        complexities[dataset][classifier] = np.mean(v)
        print(dataset, classifier, np.mean(v))

In [ ]:
sns.set_theme(font_scale=1.3)

x = []
y = []
hue = []
style = []
for dataset in data_sets:
    if dataset == 'housing':
        continue
    for classifier in classifiers:
        x.append(complexities[dataset][classifier])
        y.append(accuracies[dataset][classifier])
        hue.append(dataset)
        style.append(classifier)
        
fig, ax = plt.subplots(1, 1, figsize=(6, 3))
ax = sns.scatterplot(x, y, hue=hue, style=style, s=200)

handles, labels = ax.get_legend_handles_labels()
plt.legend(ncol=1, 
           bbox_to_anchor = (1., 1.03),
           handles=[handles[i] for i in [3, 0, 2, 1, 4, 6, 5, 7]], 
           labels=['Iris', 'Income', 'Credit', 'Travel', 'GAM', 'GBTree', 'RF', 'KNN'],
           frameon=True,
           fontsize=12,
           markerscale = 1.8)

plt.ylabel('Accuracy')
plt.xlabel('Average Degree of Variable Interaction in Shapley-GAM')
ax.set_xticks([1,2,3,4,5])
ax.set_xlim([0.85,5])
ax.set_ylim([0.6, 1.025])
ax.set_yticks([0.6, 0.7, 0.8, 0.9, 1.0])
plt.tight_layout()
plt.savefig(f'../../figures/accuracy_interaction.svg')
plt.show()